<center> <h1>ipywebrtc</h1> </center>
<center> <h1>MediaImageRecorder</h1> </center>

In [ ]:
import ipywidgets as widgets
from ipywebrtc import MediaImageRecorder, VideoStream

In [ ]:
video = VideoStream.from_file('ipyvolume.mp4')
video

## Using the image recorder, you can take screenshot of the stream clicking the camera button

In [ ]:
image_recorder = MediaImageRecorder(stream=video)
image_recorder

In [ ]:
# or programatically
image_recorder.grab()

In [ ]:
# the data is PNG encoded (by default), so we show how to use PIL to read in the data
import PIL.Image
import PIL.ImageFilter
import io
im = PIL.Image.open(io.BytesIO(image_recorder.image.value))

In [ ]:
# you could do some filtering
im.filter(PIL.ImageFilter.BLUR)

In [ ]:
# or convert them to a numpy array for manipulation
import numpy as np
im_array = np.array(im)
im_array

In [ ]:
# such as changing the channel order
PIL.Image.fromarray(im_array[...,::-1])

In [ ]:
# or a more sophisticated app using skimage
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage.color import rgb2gray
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters


image = widgets.Image()

filter_options = [('roberts', roberts), ('sobel', sobel), ('scharr', scharr), ('prewitt', prewitt)]
filter_widget = widgets.ToggleButtons(options=filter_options)

def update_image(change):
    # turn into nparray
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3] # no alpha
    
    # filter
    filter_function = filter_widget.value
    im_array_edges = adapt_rgb(each_channel)(filter_function)(im_array)
    im_array_edges = ((1-im_array_edges) * 255).astype(np.uint8)
    im_out = PIL.Image.fromarray(im_array_edges)
    
    # store in image widget
    f = io.BytesIO()
    im_out.save(f, format='png')
    image.value = f.getvalue()

image_recorder.image.observe(update_image, 'value')
filter_widget.observe(update_image, 'value')
widgets.jslink((image_recorder.image, 'width'), (image, 'width'))
widgets.jslink((image_recorder.image, 'height'), (image, 'height'))
widgets.VBox([filter_widget, video, widgets.HBox([image_recorder, image]), ])

In [ ]:
image_recorder.grab()